In [2]:
import tensorflow as tf
import gym
import numpy as np
import datetime

# from rl_agents.env_utils import rollouts_generator, get_adv_vtarg, get_gaeadv_vtarg
from rl_agents.vpg.agent import VPG_Agent
from rl_agents.ppo.agent import PPO_Agent
from rl_agents.training.buffers import GAE_Buffer
from rl_agents.training.sensei import Sensei, ExperimentRunner
from rl_agents.utils import get_actor_critic, simple_run

# for auto-reloading external modules
# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
%load_ext autoreload
%autoreload 2

tf.random.set_seed(0)
tf.keras.backend.set_floatx('float32')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Create GYM environment
Use Pendulum-v0 for now

In [3]:
# env_fn = lambda: gym.make('MountainCarContinuous-v0')
env_fn = lambda: gym.make('Pendulum-v0')
# env_fn = lambda: gym.make('MountainCar-v0')
# env_fn = lambda: gym.make('CartPole-v0')
# env_fn = lambda: gym.make('LunarLanderContinuous-v2')
# env_fn = lambda: gym.make('Acrobot-v1')
env = env_fn()
# continuous = check_if_continuous(env)
obs_dim = env.observation_space.shape
act_dim = env.action_space.shape or env.action_space.n

In [16]:
act_dim

(1,)

In [8]:
buff_vpg = GAE_Buffer(obs_dim, act_dim, 4096, gamma=0.99, lam=0.95)
actor_vpg, critic_vpg = get_actor_critic(env)
jen_vpg = VPG_Agent(actor_vpg, critic_vpg)

lam = 0.95
gamma = 0.99
epochs_actor = 1
epochs_critic = 80
sensei_vpg = Sensei(jen_vpg, env_fn, buff_vpg,
                    epochs_actor=epochs_actor, epochs_critic=epochs_critic,
                    gamma=gamma, gae_lambda=lam,
                    log_dir='logs')

In [9]:
num_ite = 200
sensei_vpg.train(num_ite, batch_size=128)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0 -1790.05227163

In [4]:
buff_ppo = GAE_Buffer(obs_dim, act_dim, 2048, gamma=0.99, lam=0.95)
actor_ppo, critic_ppo = get_actor_critic(env)
jen_ppo = PPO_Agent(actor_ppo, critic_ppo, act_dim)

lam = 0.95
gamma = 0.99
epochs_actor = 10
epochs_critic = 80
sensei_ppo = Sensei(jen_ppo, env_fn, buff_ppo,
                    epochs_actor=epochs_actor, epochs_critic=epochs_critic,
                    gamma=gamma, gae_lambda=lam,
                    log_dir='logs')

In [5]:
num_ite = 200
sensei_ppo.train(num_ite, batch_size=128)


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.


If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.

0 20.63917525773

# Testing buffer

In [ ]:
a = tf.keras.optimizers.Adam(3e-3)

In [ ]:
a

In [ ]:
buff = GAE_Buffer(obs_dim, act_dim, 20, gamma=0.99, lam=0.95)

In [ ]:
actor_test, critic_test = get_actor_critic(env)

In [ ]:
test_vpg = VPG_Agent(actor_test, critic_tes, act_dim)

In [ ]:
test_runner = ExperimentRunner(test_vpg, env, buff)
test_runner.num_ite = 1

In [ ]:
for i, rollout in enumerate(test_runner):
    print(i)

In [ ]:
env = gym.make('Pendulum-v0')
obs_dim = env.observation_space.shape
act_dim = env.action_space.shape or env.action_space.n

actor_vpg, critic_vpg = get_actor_critic(env)
jen_vpg = VPG_Agent(actor_vpg, critic_vpg, act_dim)

simple_run(env, jen_vpg)

In [ ]:
buff_ppo = GAE_Buffer(obs_dim, act_dim, 2048, gamma=0.99, lam=0.95)
actor_ppo, critic_ppo = get_actor_critic(env)
jen_ppo = PPO_Agent(actor_ppo, critic_ppo, act_dim)

lam = 0.95
gamma = 0.99
epochs_actor = 10
epochs_critic = 80
sensei_ppo = Sensei(jen_ppo, env_fn, buff_ppo,
                    epochs_actor=epochs_actor, epochs_critic=epochs_critic,
                    gamma=gamma, gae_lambda=lam,
                    log_dir='logs')

In [ ]:
num_ite = 200
sensei_ppo.train(num_ite, batch_size=256)

# Vanilla Policy Gradient

## Initialization

In [ ]:
actor_vpg = GaussianActor(obs_dim, act_dim) if is_continuous else CategoricalActor(obs_dim, act_dim)
critic_vpg = Critic(obs_dim)
jen_vpg = VPG_Agent(actor_vpg, critic_vpg, is_continuous, act_dim)
generator_vpg = rollouts_generator(jen_vpg, env, is_continuous, horizon=2048)

alg_name = "VPG"
lam = 0.95
gamma = 0.99
epochs_actor = 1
epochs_critic = 40
sensei_vpg = Sensei(jen_vpg, alg_name, env_fn,
                    horizon=2048, epochs_actor=epochs_actor, epochs_critic=epochs_critic,
                    gamma=gamma, gae_lambda=lam,
                    log_dir='logs')

# Training loop

In [ ]:
num_ite = 100
sensei_vpg.train(num_ite, batch_size=256)

In [ ]:
num_ite = 50
sensei_vpg.train(num_ite, record=False, batch_size=256)

In [ ]:
def discount_cumsum(x, discount):
    """
    magic from rllab for computing discounted cumulative sums of vectors.
    input: 
        vector x, 
        [x0, 
         x1, 
         x2]
    output:
        [x0 + discount * x1 + discount^2 * x2,  
         x1 + discount * x2,
         x2]
    """
    return scipy.signal.lfilter([1], [1, float(-discount)], x[::-1], axis=0)[::-1]

In [ ]:
rollout = generator_ppo.__next__()

In [ ]:
gae, td = get_gaeadv_vtarg(rollout, 0.95, 0.99)

In [ ]:
rews, vals = rollout["rew"][:198], rollout["vpred"][:198]
rews = np.append(rews, rollout["vpred"])
vals = np.append(vals, rollout["next_vpred"])
deltas = rews[:-1] + 0.99 * vals[1:] - vals[:-1]
gae2 = discount_cumsum(deltas, 0.99*0.95)

In [ ]:
np.sum(gae2 - gae)

In [ ]:
gae2

In [ ]:
gae

In [ ]:
gae-gae2

In [ ]:
rollout["new"][:198]

In [ ]:
act_dim

In [ ]:
jen_vpg.actor_step

In [ ]:
env.action_space.sample()

In [ ]:
b = np.array([1,2])
b[:1]

In [ ]:
b[-1:]

In [ ]:
limits = np.array([[-5, 5], [-10, 10]])
ac = np.array([2, -19])

print(limits[0,:])
np.clip(ac, limits[:, 0], limits[:, 1])